# Facefusion on Sagemaker

## build image

In [ ]:
# Build an image that can do training and inference in SageMaker
# This is a Python 3 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

# FROM pytorch/pytorch:2.1.0-cuda11.8-cudnn8-devel
# FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:2.1.0-cpu-py310-ubuntu20.04-ec2
# FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:2.1.0-gpu-py310-cu118-ubuntu20.04-ec2
# FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.13.1-gpu-py39-cu117-ubuntu20.04-ec2
# ref from https://github.com/facefusion/facefusion-docker
FROM python:3.10
ARG DEBIAN_FRONTEND=noninteractive
ARG FACEFUSION_VERSION=2.3.0
ENV GRADIO_SERVER_NAME=0.0.0.0
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE
ENV PATH="/opt/program:${PATH}"


WORKDIR /opt/program

RUN apt-get update
RUN apt-get install curl -y
RUN apt-get install ffmpeg -y

##安装sagemaker endpoint所需的组件
RUN apt-get install nginx -y  
RUN pip install --no-cache-dir boto3 flask gunicorn
# RUN git clone https://github.com/facefusion/facefusion.git --branch ${FACEFUSION_VERSION} --single-branch .
##拷贝包含sagemaker endpoint所需的python和配置文件
COPY facefusion /opt/program
RUN python install.py --torch cpu --onnxruntime default

WORKDIR /opt/program

In [ ]:
# 在 Facefusion-Sagemaker-Studio-Lab 目录执行如下命令，如上docker file是已CPU举例的，可以修改使用GPU 可以参考gpu_Dockerfile
!./build_and_push.sh faces-swap-on-sagemaker

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  15.25MB
Step 1/22 : FROM nvcr.io/nvidia/pytorch:23.06-py3
 ---> e265317f0fb8
Step 2/22 : ARG FACEFUSION_VERSION=2.4.1
 ---> Using cache
 ---> cbce6337d919
Step 3/22 : ENV TZ=Etc/UTC
 ---> Using cache
 ---> c8a3bd11bd12
Step 4/22 : ENV GRADIO_SERVER_NAME=0.0.0.0
 ---> Using cache
 ---> ab75e5d893f6
Step 5/22 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> b5ca82ea5edc
Step 6/22 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> f390c0f07078
Step 7/22 : ENV PATH=/opt/program:/usr/local/cuda:${PATH}
 ---> Using cache
 ---> fb4d4ac63324
Step 8/22 : WORKDIR /opt/program
 ---> Using cache
 ---> 755a781aa7ac
Step 9/22 : COPY ./ /opt/program
 ---> 0badf6631443
Step 10/22 : RUN apt-get install git -y
 --->

In [ ]:
import boto3
import sagemaker
from sagemaker import Model, image_uris, serializers, deserializers

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment
bucket = sess.default_bucket()
image="faces-swap-on-sagemaker"
s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

full_image_uri=f"{account_id}.dkr.ecr.{region}.amazonaws.com/{image}:latest"
print(full_image_uri)

## remote debug test

In [ ]:
!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'facefusion')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'facefusion')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

In [ ]:
from sagemaker_ssh_helper.wrapper import SSHModelWrapper
model = Model(image_uri=full_image_uri, model_data=model_data, role=role,dependencies=[SSHModelWrapper.dependency_dir()] )

In [ ]:
from sagemaker_ssh_helper.wrapper import SSHModelWrapper
instance_type = "ml.g5.xlarge"
endpoint_name = sagemaker.utils.name_from_base("facefusion-byoc")


ssh_wrapper = SSHModelWrapper.create(model, connection_wait_time_seconds=0)  # <--NEW--

predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    wait=True
)

In [ ]:
instance_ids = ssh_wrapper.get_instance_ids(timeout_in_sec=0)  # <--NEW-- 
print(f"To connect over SSM run: aws ssm start-session --target {instance_ids[0]}")

## SM endpoint test

### create sagemaker model

In [ ]:
import boto3
import re
import os
import json
import uuid
import boto3
import sagemaker
from time import gmtime, strftime
## for debug only
from sagemaker_ssh_helper.wrapper import SSHModelWrapper
sm_client = boto3.client(service_name='sagemaker')



def create_model():
    image=full_image_uri
    model_name="facefusion-sagemaker-01"+strftime("%Y-%m-%d-%H-%M-%S", gmtime())
    create_model_response = sm_client.create_model(
        ModelName=model_name,
        ExecutionRoleArn=role,
        Containers=[{"Image": image}],
    )
    print(create_model_response)
    return model_name

In [ ]:
model_name=create_model()

### create endpoint configuration

In [ ]:
endpointConfigName = "facefusion-sagemaker-configuration"+strftime("%Y-%m-%d-%H-%M-%S", gmtime())
def create_endpoint_configuration():
    create_endpoint_config_response = sm_client.create_endpoint_config(     
        EndpointConfigName=endpointConfigName,
        ProductionVariants=[
            {
                "ModelName":"facefusion-sagemaker-012024-03-28-04-00-03",
                #"ModelName":model_name,
                "VariantName": "facefusion-sagemaker"+"-variant",
                "InstanceType": "ml.g5.2xlarge",  # 指定 g5.2xlarge 机器
                "InitialInstanceCount": 1,
                "ModelDataDownloadTimeoutInSeconds": 1200,
                "ContainerStartupHealthCheckTimeoutInSeconds": 1200
            }
        ],
    )
    print(create_endpoint_config_response)
    return endpointConfigName

In [ ]:
create_endpoint_configuration()

### create endpoint

In [ ]:
endpointName="facefusion-sagemaker-endpoint"+strftime("%Y-%m-%d-%H-%M-%S", gmtime())
def create_endpoint():
    create_endpoint_response = sm_client.create_endpoint(
        EndpointName=endpointName,
        #EndpointConfigName="facefusion-sagemaker-configuration2024-03-28-04-03-53",
        EndpointConfigName=endpointConfigName
    )
    print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])
    resp = sm_client.describe_endpoint(EndpointName=endpointName)
    print("Endpoint Status: " + resp["EndpointStatus"])
    print("Waiting for {} endpoint to be in service".format("facefusion-sagemaker-endpoint"))
    waiter = sm_client.get_waiter("endpoint_in_service")
    waiter.wait(EndpointName=endpointName)

In [ ]:
create_endpoint()

## Realtime inferecne with sagemaker endpoint

In [ ]:
import json
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")
#endpointName="facefusion-sagemaker-endpoint2024-04-03-23-49-44"
request = {"method":"submit","input":['-s','s3://sagemaker-us-west-2-687912291502/images/image1.jpg',
                                      '-t','s3://sagemaker-us-west-2-687912291502/video/test.mp4',
                                      '-o','/tmp/','-u','s3://sagemaker-us-west-2-687912291502/video/test_out2.mp4',
                                      '--headless'],}
def invoke_endpoint():
    content_type = "application/json"
    request_body = request
    payload = json.dumps(request_body)
    print(payload)
    response = runtime_sm_client.invoke_endpoint(
        EndpointName=endpointName,
        ContentType=content_type,
        Body=payload,
    )
    result = response['Body'].read().decode()
    print('返回：',result)

In [ ]:
response=invoke_endpoint()

In [ ]:
!aws s3 cp s3://sagemaker-us-west-2-687912291502/video/test_out2.mp4 ./

## Async inference

In [ ]:
_time_tag = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
_variant_name =  'facusion-'+ _time_tag
endpoint_config_name = f'facefusion-{str(uuid.uuid4())}'

response = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': _variant_name,
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.c5.large',
            'InitialVariantWeight': 1
        },
    ]
    ,
    AsyncInferenceConfig={
        'OutputConfig': {
            'S3OutputPath': f's3://{bucket}/stablediffusion/asyncinvoke/out/'
        }
    }
)

In [ ]:
endpoint_name = f'facefusion-{str(uuid.uuid4())}'

response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName="endpoint_config_name",
    
)

print(f'终端节点:{endpoint_name} 正在创建中，首次启动中会加载模型，请耐心等待, 请在控制台上查看状态')


In [ ]:
import time
def predict_async(endpoint_name,payload):
    runtime_client = boto3.client('runtime.sagemaker')
    input_file=str(uuid.uuid4())+".json"
    s3_resource = boto3.resource('s3')
    s3_object = s3_resource.Object(bucket, f'stablediffusion/asyncinvoke/input/{input_file}')
    payload_data = json.dumps(payload).encode('utf-8')
    s3_object.put( Body=bytes(payload_data))
    input_location=f's3://{bucket}/stablediffusion/asyncinvoke/input/{input_file}'
    print(f'input_location: {input_location}')
    response = runtime_client.invoke_endpoint_async(
        EndpointName=endpoint_name,
        InputLocation=input_location
    )
    result =response.get("OutputLocation",'')
    wait_async_result(result)
    
def wait_async_result(output_location,timeout=60):
    current_time=0
    while current_time<timeout:
        if s3_object_exists(output_location):
            print("have async result")
            draw_image(output_location)
            break
        else:
            time.sleep(5)
def s3_object_exists(s3_path):
    """
    s3_object_exists
    """
    try:
        s3 = boto3.client('s3')
        base_name=os.path.basename(s3_path)
        _,ext_name=os.path.splitext(base_name)
        bucket,key=get_bucket_and_key(s3_path)
        
        s3.head_object(Bucket=bucket, Key=key)
        return True
    except Exception as ex:
        print("job is not completed, waiting...")   
        return False

## client lib test

In [ ]:
from ModelClient import ModelClient
client = ModelClient("facefusion-v2.2")
client.set_endpoint("facefusion-byoc-2024-04-25-13-23-03-735")
job_id=client.submit_job("test01",swap_face_image_s3_path="s3://sagemaker-us-west-2-687912291502/images/image1.jpg",
                           source_video_s3_path='s3://sagemaker-us-west-2-687912291502/video/test.mp4',
                           output_video_s3_dir='s3://sagemaker-us-west-2-687912291502/video')




In [ ]:
from ModelClient import ModelClient
job_id="f0cbd696ab674fa0be57cc859169fe16-20240425133629"
client = ModelClient("facefusion-v2.2")
client.set_endpoint("facefusion-sagemaker-endpoint2024-04-24-04-54-14")
status = client.get_status( "test01", job_id)
status

In [ ]:
response = client.get_result(job_id)

In [ ]:
!aws s3 ls s3://sagemaker-us-west-2-687912291502/video/